## 1. Simple Multi-Threaded Web Scraper

In [1]:
from concurrent.futures import ThreadPoolExecutor
import time
import requests
from typing import List

print("=" * 50)
print("MULTI-THREADED WEB SCRAPER")
print("=" * 50)

# Simulated fetch (use real URLs in practice)
def fetch_page(url):
    """Fetch and process a single URL"""
    print(f"Fetching {url}...")
    # In real code: response = requests.get(url, timeout=5)
    time.sleep(1)  # Simulate network delay
    return {
        'url': url,
        'status': 200,
        'content_length': 5000
    }

def scrape_urls(urls: List[str], max_workers: int = 5):
    """Scrape multiple URLs concurrently"""
    results = []
    
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_page, url) for url in urls]
        
        for future in futures:
            try:
                result = future.result(timeout=10)
                results.append(result)
            except Exception as e:
                print(f"Error: {e}")
    
    elapsed = time.time() - start_time
    return results, elapsed

# Test
urls = [f'http://example.com/page{i}' for i in range(10)]
results, elapsed = scrape_urls(urls, max_workers=5)

print(f"\nScraped {len(results)} pages in {elapsed:.2f}s")
print(f"Sequential would take ~{len(urls)}s")
print(f"Speedup: {len(urls) / elapsed:.1f}x")

MULTI-THREADED WEB SCRAPER
Fetching http://example.com/page0...
Fetching http://example.com/page1...
Fetching http://example.com/page2...
Fetching http://example.com/page3...
Fetching http://example.com/page4...
Fetching http://example.com/page5...Fetching http://example.com/page6...
Fetching http://example.com/page7...
Fetching http://example.com/page8...
Fetching http://example.com/page9...


Scraped 10 pages in 2.02s
Sequential would take ~10s
Speedup: 4.9x


## 2. Scraper with Queue

import threading
from queue import Queue
import time

print("\n" + "=" * 50)
print("SCRAPER WITH QUEUE")
print("=" * 50)

class QueueBasedScraper:
    def __init__(self, num_workers=5):
        self.url_queue = Queue()
        self.results = []
        self.lock = threading.Lock()
        self.num_workers = num_workers
    
    def fetch_url(self, url):
        print(f"Fetching {url}...")
        time.sleep(1)
        return {'url': url, 'status': 200}
    
    def worker(self):
        while True:
            url = self.url_queue.get()
            if url is None:  # Poison pill
                break
            
            try:
                result = self.fetch_url(url)
                with self.lock:
                    self.results.append(result)
            finally:
                self.url_queue.task_done()
    
    def scrape(self, urls):
        # Add URLs to queue
        for url in urls:
            self.url_queue.put(url)
        
        # Create workers
        threads = []
        for _ in range(self.num_workers):
            t = threading.Thread(target=self.worker)
            t.start()
            threads.append(t)
        
        # Wait for queue to empty
        self.url_queue.join()
        
        # Add poison pills
        for _ in range(self.num_workers):
            self.url_queue.put(None)
        
        # Wait for workers
        for t in threads:
            t.join()
        
        return self.results

# Test
scraper = QueueBasedScraper(num_workers=3)
urls = [f'http://example.com/page{i}' for i in range(6)]
results = scraper.scrape(urls)
print(f"\nScraped {len(results)} pages")

In [2]:
import threading
from queue import Queue
import time

print("\n" + "=" * 50)
print("SCRAPER WITH QUEUE")
print("=" * 50)

class QueueBasedScraper:
    def __init__(self, num_workers=5):
        self.url_queue = Queue()
        self.results = []
        self.lock = threading.Lock()
        self.num_workers = num_workers
    
    def fetch_url(self, url):
        print(f"Fetching {url}...")
        time.sleep(1)
        return {'url': url, 'status': 200}
    
    def worker(self):
        while True:
            url = self.url_queue.get()
            if url is None:  # Poison pill
                break
            
            try:
                result = self.fetch_url(url)
                with self.lock:
                    self.results.append(result)
            finally:
                self.url_queue.task_done()
    
    def scrape(self, urls):
        # Add URLs to queue
        for url in urls:
            self.url_queue.put(url)
        
        # Create workers
        threads = []
        for _ in range(self.num_workers):
            t = threading.Thread(target=self.worker)
            t.start()
            threads.append(t)
        
        # Wait for queue to empty
        self.url_queue.join()
        
        # Add poison pills
        for _ in range(self.num_workers):
            self.url_queue.put(None)
        
        # Wait for workers
        for t in threads:
            t.join()
        
        return self.results

# Test
scraper = QueueBasedScraper(num_workers=3)
urls = [f'http://example.com/page{i}' for i in range(6)]
results = scraper.scrape(urls)
print(f"\nScraped {len(results)} pages")


SCRAPER WITH QUEUE
Fetching http://example.com/page0...
Fetching http://example.com/page1...
Fetching http://example.com/page2...
Fetching http://example.com/page3...
Fetching http://example.com/page4...
Fetching http://example.com/page5...

Scraped 6 pages
